# MGSC 416: GROUP PROJECT
<u>GROUP 11</u>: Alex DaSilva (260967146), Serena Yu (260948951), Tiphaine Levan (261079401), David Ming (260904192)

In [31]:
import gurobipy as gp
from gurobipy import *
import math
import numpy as np
import pandas as pd

### <span style='color:red'>**PROBLEM FORMULATION**</span>

<span style='color:maroon'> **DECISION VARIABLES**</span>
* *$w$* = stock's weight in the portfolio *(continous)*
* *$x$* = chosen stock *(binary)*

<span style='color:maroon'> **DEFINED VARIABLES**</span>
* *$α$* = stock's alpha
* *$v$* = stock's value-at-risk
* *$d$* = stock's downside deviation
* *$p$* = stock's CAPM portfolio volatility
* *$s$* = stock's sector
* *$i, j$* = panel data identifier
 * written as "coordinates", where $i$ represents the week and $j$ represents the stock

<span style='color:maroon'>**OBJECTIVE: MAXIMIZE EXCESS RETURNS**</span>
$$\sum_{j=1}^{452}(w_{ij}*α_{ij}),\ \forall{i}$$

<span style='color:maroon'>**SUBJECT TO**</span>
1. A stock is either chosen or not.
$$x_{ij}\in{}\{0,1\}$$


2. In total, 25 stocks must be chosen.
$$\sum_{j=1}^{452}x_{ij}=25,\ \forall{i}$$


3. The total weight for the chosen stocks in the portfolio must equal to 100%.
$$\sum_{j=1}^{452}w_{ij}=1,\ \forall{i}$$


4. Each chosen stock's weight in the portfolio must be between 1% and 100% (both inclusive)
$$0.01\leq w_{i,j} \leq1$$


5. The total value-at-risk of the portfolio must be greater than or equal to -3.5%.
$$\sum_{j=1}^{452}(w_{i,j} * v_{i,j})\geq-3.5,\ \forall{i}$$


6. The total downside deviation of the portfolio must be less than or equal to 3%.
$$\sum_{j=1}^{452}(w_{i,j} * d_{i,j})\leq3,\ \forall{i}$$


5. The total CAPM portfolio volatility of the portfolio must be less than or equal to 4\%.
$$\sum_{j=1}^{452}(w_{i,j} * p_{i,j})\leq4,\ \forall{i}$$


7. The sector weight for each sector of the portfolio must be less than or equal to 20%.
$$\sum_{j=1}^{452}(w_{ij}*s_{ij})\leq0.2,\ \forall{i}$$

### <span style='color:red'>**DATA PREPARATION**</span>

In [32]:
df = pd.read_csv('Weekly416DataFinal.csv')

# dummifying the sectors
df = pd.concat([df, pd.get_dummies(df['Sector'])], axis=1)

# setting multi-index
df.set_index(['Week', 'symbol'], inplace=True)

# unique weeks
week_list = df.index.get_level_values('Week').unique()

# unique stocks
stock_list = df.index.get_level_values('symbol').unique()

df.head(5)

Weekly_Returns  Weekly_LogReturns  Weekly_Volatility  \
Week       symbol                                                         
2021-01-15 A            -0.189797          -0.671220           1.611180   
           AAL           6.342780           4.079556           8.346176   
           AAPL         -1.586812          -3.789162           3.437362   
           ABBV          3.222188           4.147547           2.896805   
           ABT           0.080932           0.123926           4.540354   

                   MktPrem   SMB   HML     RF  MktPrem_Vol   SMB_Vol  \
Week       symbol                                                      
2021-01-15 A         -1.03  2.33  2.15  0.001     1.089105  2.744194   
           AAL       -1.03  2.33  2.15  0.001     1.089105  2.744194   
           AAPL      -1.03  2.33  2.15  0.001     1.089105  2.744194   
           ABBV      -1.03  2.33  2.15  0.001     1.089105  2.744194   
           ABT       -1.03  2.33  2.15  0.001     1.089105  2.744194   

                    HML_Vol  ...  Consumer Staples  Energy  Financials  \
Week       symbol            ...                                         
2021-01-15 A       2.228206  ...             False   False       False   
           AAL     2.228206  ...             False   False       False   
           AAPL    2.228206  ...             False   False       False   
           ABBV    2.228206  ...             False   False       False   
           ABT     2.228206  ...             False   False       False   

                   Health Care  Industrials Information Technology  Market  \
Week       symbol                                                            
2021-01-15 A              True        False                  False   False   
           AAL           False         True                  False   False   
           AAPL          False        False                   True   False   
           ABBV           True        False                  False   False   
           ABT            True        False                  False   False   

                   Materials  Real Estate  Utilities  
Week       symbol                                     
2021-01-15 A           False        False      False  
           AAL         False        False      False  
           AAPL        False        False      False  
           ABBV        False        False      False  
           ABT         False        False      False  

[5 rows x 35 columns]

In [33]:
# Getting the Constraint Means
print(df['Weekly_VaR'].mean())
print(df['ddev'].mean())
pvar = ((df['MarketBeta']**2 * df['MktPrem_Vol']**2 + df['IdioVol']**2))**0.5
print(pvar.mean())

-3.904400075532876
3.304808352822111
3.873555035307403


### <span style='color:red'>**MODEL CREATION**</span>

In [34]:
m = Model('portfolio')

# decision variables
x = m.addVars(df.index, vtype=GRB.BINARY, name='x')
w = m.addVars(df.index, vtype=GRB.CONTINUOUS, name='w')

# set objective function
m.setObjective(quicksum(w[i,j] * df.loc[(i,j), 'alpha'] for i,j in df.index), GRB.MAXIMIZE)

In [35]:
# -- CONSTRAINTS --

# empty dictionary for three constraints to store their slack values
constr3 = {'var': {}, 'dd': {}, 'pv': {}}

for i in week_list:
    # choose 25 stocks
    m.addConstr(quicksum(x[i,j] for j in stock_list) == 25, f'choose_25_stocks_{i}')
    
    # total weight for chosen stocks = 100%
    m.addConstr(quicksum(w[i,j] for j in stock_list) == 1, f'total_weight_{i}')
    
    # individual stock weight
    for j in stock_list:
        m.addConstr(w[i,j] >= 0.01 * x[i,j], f'individual_weight_lower_{i}_{j}')
        m.addConstr(w[i,j] <= 1 * x[i,j], f'individual_weight_upper_{i}_{j}')
          
    # value-at-risk
    constr3['var'][i] = m.addConstr(quicksum(w[i,j] * df.loc[(i,j), 'Weekly_VaR'] for j in stock_list) >= -3.9044, f'VaR_constraint_{i}_{j}')
    
    # downside deviation
    ddev = quicksum(w[i,j] * df.loc[(i,j), 'ddev'] for j in stock_list)
    constr3['dd'][i] = m.addConstr(ddev <= 3.3048, f'Ddev_constraint_{i}')
    
    # CAPM portfolio volatility
    portvol = quicksum(w[i,j] * (df.loc[(i,j), 'MarketBeta']**2 * df.loc[(i,j), 'MktPrem_Vol']**2 + df.loc[(i,j), 'IdioVol']**2)**0.5 for j in stock_list)
    constr3['pv'][i] = m.addConstr(portvol <= 3.87355, f'PortVol_constraint_{i}') #Consistently Binding   
    
    # sector weight
    sector_list = df['Sector'].unique()
    for sector in sector_list:
        sector_weight = quicksum(df.loc[(i,j), sector] * w[i,j] for j in stock_list)
        sec = m.addConstr(sector_weight <= 0.2)    

In [36]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 142755 rows, 140120 columns and 697346 nonzeros
Model fingerprint: 0xd7d90dd2
Variable types: 70060 continuous, 70060 integer (70060 binary)
Coefficient statistics:
  Matrix range     [2e-06, 7e+01]
  Objective range  [9e-06, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 2e+01]
Presolve removed 141835 rows and 139216 columns
Presolve time: 1.89s
Presolved: 920 rows, 904 columns, 4516 nonzeros
Variable types: 452 continuous, 452 integer (452 binary)
Found heuristic solution: objective 1161.3079707

Root relaxation: objective 1.166278e+03, 558 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1166.27812    0   

In [37]:
# print chosen stocks for each week
print('-- CHOSEN STOCKS --','\n')
for i in week_list:
    for j in stock_list:
        if w[i,j].x >= 0.000001:
            print(f"Week: {i}, Stock: {j}, Weight: {round(w[i,j].x,4)}")

-- CHOSEN STOCKS -- 

Week: 2021-01-15, Stock: AES, Weight: 0.19
Week: 2021-01-15, Stock: AMP, Weight: 0.01
Week: 2021-01-15, Stock: BXP, Weight: 0.01
Week: 2021-01-15, Stock: CINF, Weight: 0.01
Week: 2021-01-15, Stock: CTRA, Weight: 0.18
Week: 2021-01-15, Stock: DHI, Weight: 0.01
Week: 2021-01-15, Stock: EQR, Weight: 0.066
Week: 2021-01-15, Stock: FE, Weight: 0.01
Week: 2021-01-15, Stock: GM, Weight: 0.164
Week: 2021-01-15, Stock: HPE, Weight: 0.2
Week: 2021-01-15, Stock: JCI, Weight: 0.01
Week: 2021-01-15, Stock: KMI, Weight: 0.01
Week: 2021-01-15, Stock: L, Weight: 0.01
Week: 2021-01-15, Stock: LH, Weight: 0.01
Week: 2021-01-15, Stock: LMT, Weight: 0.01
Week: 2021-01-15, Stock: LOW, Weight: 0.01
Week: 2021-01-15, Stock: MET, Weight: 0.01
Week: 2021-01-15, Stock: MRNA, Weight: 0.01
Week: 2021-01-15, Stock: NOC, Weight: 0.01
Week: 2021-01-15, Stock: OKE, Weight: 0.01
Week: 2021-01-15, Stock: PFG, Weight: 0.01
Week: 2021-01-15, Stock: PRU, Weight: 0.01
Week: 2021-01-15, Stock: SYF, Wei

In [38]:
# print slack values for 'var', 'dd', 'pv' for each week
for constraint, week_constr in constr3.items():
    print(f'-- CONSTRAINT: {constraint} --')
    for week, constr in week_constr.items():
        slack = constr.slack
        print(f'{week}: slack = {slack}')
    print('\n')

-- CONSTRAINT: var --
2021-01-15: slack = -2.5960952926128456
2021-01-22: slack = -5.0285001968581
2021-01-29: slack = -2.8301023039591535
2021-02-05: slack = -5.786976384593597
2021-02-12: slack = -4.891229259075647
2021-02-19: slack = -5.03500972401859
2021-02-26: slack = -3.0512959660374572
2021-03-05: slack = -4.506742755474794
2021-03-12: slack = -4.1202914040088645
2021-03-19: slack = -3.1039698280463934
2021-03-26: slack = -2.611474333568028
2021-04-01: slack = -3.005559582055346
2021-04-09: slack = -2.8560398666889872
2021-04-16: slack = -3.3963908283662185
2021-04-23: slack = -3.7437559875833935
2021-04-30: slack = -2.719327906776291
2021-05-07: slack = -5.769678312601455
2021-05-14: slack = -2.314214058992172
2021-05-21: slack = -2.9481450487768055
2021-05-28: slack = -3.169192889212067
2021-06-04: slack = -4.184571924084324
2021-06-11: slack = -3.4399264896374007
2021-06-18: slack = -2.9788919025845395
2021-06-25: slack = -4.901302391805525
2021-07-02: slack = -2.31800529744

In [39]:
# add the model's results to the original datasets as two new columns: Chosen (binary), Weight
chosen_status = []
weights = []

for i in week_list:
    for j in stock_list:
        # is the stock chosen for the current week (i)
        chosen = x[i,j].x if (i,j) in x else 0
        chosen_status.append(chosen)
        
        # if chosen, get the stock's weight; otherwise weight=0
        if chosen:
            #weight = round(w[i,j].x,4) if (i,j) in w else 0
            weight = w[i,j].x if (i,j) in w else 0
        else:
            weight = 0
        weights.append(weight)

# add the lists as columns to the original dataframe
df['Chosen'] = chosen_status
df['Weight'] = weights

df.head(5)

Weekly_Returns  Weekly_LogReturns  Weekly_Volatility  \
Week       symbol                                                         
2021-01-15 A            -0.189797          -0.671220           1.611180   
           AAL           6.342780           4.079556           8.346176   
           AAPL         -1.586812          -3.789162           3.437362   
           ABBV          3.222188           4.147547           2.896805   
           ABT           0.080932           0.123926           4.540354   

                   MktPrem   SMB   HML     RF  MktPrem_Vol   SMB_Vol  \
Week       symbol                                                      
2021-01-15 A         -1.03  2.33  2.15  0.001     1.089105  2.744194   
           AAL       -1.03  2.33  2.15  0.001     1.089105  2.744194   
           AAPL      -1.03  2.33  2.15  0.001     1.089105  2.744194   
           ABBV      -1.03  2.33  2.15  0.001     1.089105  2.744194   
           ABT       -1.03  2.33  2.15  0.001     1.089105  2.744194   

                    HML_Vol  ...  Financials  Health Care  Industrials  \
Week       symbol            ...                                         
2021-01-15 A       2.228206  ...       False         True        False   
           AAL     2.228206  ...       False        False         True   
           AAPL    2.228206  ...       False        False        False   
           ABBV    2.228206  ...       False         True        False   
           ABT     2.228206  ...       False         True        False   

                   Information Technology  Market Materials  Real Estate  \
Week       symbol                                                          
2021-01-15 A                        False   False     False        False   
           AAL                      False   False     False        False   
           AAPL                      True   False     False        False   
           ABBV                     False   False     False        False   
           ABT                      False   False     False        False   

                   Utilities  Chosen  Weight  
Week       symbol                             
2021-01-15 A           False     0.0     0.0  
           AAL         False     0.0     0.0  
           AAPL        False     0.0     0.0  
           ABBV        False     0.0     0.0  
           ABT         False    -0.0     0.0  

[5 rows x 37 columns]

In [15]:
# saving the solution
df.to_csv('WeeklyGurobiSolutionFinal.csv', index=False)